In [ ]:
#Byte-at-a-time ECB decryption (Simple)
#importing classes
import random
from random import randint
import os
import string
# The below file is given by OPEN SSL
!pip install pycrypto
from Crypto.Cipher import AES
from math import ceil
from binascii import unhexlify, b2a_base64, a2b_base64, hexlify
import chardet

In [ ]:
def pkcs7(message, blocksize):
  padlen = blocksize-len(message)%blocksize
  if(padlen == blocksize):
    padlen = 0
  pkcsmessage=bytes([padlen])*padlen
  return message+pkcsmessage

In [ ]:
def random_byte_create(lenofbytes):
  randomnumber=[]
  for i in range(lenofbytes):
    randomnumber.append(random.randint(0,255))
  return bytes(randomnumber)


In [ ]:
Key_global=random_byte_create(16)
given_plain_text=bytes("Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK",'utf-8')


In [ ]:
def encypt_oracle(message):
  #First convert the given plain text in base 64
  base64_plaintext=a2b_base64(given_plain_text)
  # Given your-string || unknown-string
  new_message=message + base64_plaintext
  #ECB Encryption
  plaintext=pkcs7(new_message,16)
  cipher_object=AES.new(Key_global,AES.MODE_ECB)
  return cipher_object.encrypt(plaintext)

In [ ]:
def bytes_to_str(message):
    return "".join(filter(lambda x: x in string.printable, "".join(map(chr, message))))

In [ ]:
#Detect the Encryption mechanism of the block
def guess_aes_ecb(ciphertext,Block_size):
  #The logic is simple, In ECB the block content will be same , becuase for same message the cipher text will be same and not randomized
  #This formula only fails when messages are all unique
  blocks_of_ciphertext=[]
  number_of_blocks=len(ciphertext)//Block_size
  for i in range(number_of_blocks):
    blocks_of_ciphertext.append(ciphertext[i:i+Block_size])
  unique_num_blocks=set(blocks_of_ciphertext)
  #Returns True for ECB otherwise false
  return len(unique_num_blocks)<len(ciphertext)

In [ ]:
def guess_first_byte():
  #This logic contains 3 Section
  # 1 )Discover the unknownblock size of the ciphertext I have taken arbitraty number of iterations, it can be infinity also
  for x in range(200):
  #When size==0 its the block size otherwise calculate it recursively
    if x == 0:
      ciphertext_blocksize=len(encypt_oracle(b'A'*x))
      continue
    new_ciphertext_blocksize=len(encypt_oracle(b'A'*x))-ciphertext_blocksize
    if new_ciphertext_blocksize == 0:
      ciphertext_blocksize=new_ciphertext_blocksize
    else: 
      break
  
  # Check if its ECB or not with the unknown block size
  if not guess_aes_ecb(encypt_oracle(b'A'*new_ciphertext_blocksize*5),new_ciphertext_blocksize):
    return b'The Given Text is Not in ECB Mode'

  # Otherwise follow rest of the steps, find the unknown block length
  final_block_length=len(encypt_oracle(b'A'*0))//new_ciphertext_blocksize
  first_byte_string=b''
  unknown_string = b''
  input_block=b'A'* new_ciphertext_blocksize
  for block_number in range(final_block_length):
    unknown_string_block = b''
    # First new_ciphertext_blocksize-1 blocks becomes random string in A-->check one_byte_short line , then the last block will have the random UNKNOWN STRING
    # Again we do the above step but now new_ciphertext_blocksize-2 blocks becomes random string--> check one_byte_short, then the last block will have the random UNKNOWN STRING
    # This we repeat again and again 
    for n in range(new_ciphertext_blocksize-1, -1, -1):
        input_block = input_block[1:] 
        last_byte_dict = {encypt_oracle(input_block + bytes([b]))[:new_ciphertext_blocksize]: bytes([b]) for b in range(256)}
        one_byte_short = encypt_oracle(b'A'*n)[block_number*new_ciphertext_blocksize:(block_number+1)*new_ciphertext_blocksize]
        if one_byte_short not in last_byte_dict:
          break
        last_byte = last_byte_dict[one_byte_short]
        unknown_string_block += last_byte
        input_block += last_byte
    unknown_string+=unknown_string_block
    input_block=unknown_string_block
  return unknown_string



In [ ]:
if __name__== "__main__":
  for line in bytes_to_str(guess_first_byte()).split("\n"):
    print(line)

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by

